In [45]:
import openai
from latex2sympy2 import latex2sympy
import json

#model = "gpt-3.5-turbo-0613"
model = "gpt-4-0613"

def calc_latex_math_expression(expression):
    sympy_expr = latex2sympy(expression)
    result = sympy_expr.doit()
    return str(result)

functions = [
     {
      "name": "calc_latex_math_expression",
      "description": "Calculate a Latex mathematical expression",
      "parameters": {
        "type": "object",
        "properties": {
          "expression": {
            "type": "string",
            "description": "The Latex expression to calculate"
          },
        },
        "required": ["expression"]
      }
    } 
  ]

available_functions = {
    "calc_latex_math_expression": calc_latex_math_expression,
}

In [46]:
def run_conversation(content):
  messages = [
    {"role": "user", "content": content},
    ]
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  messages.append(response_message)  # extend conversation with assistant's reply

  # Step 2: check if GPT wanted to call a function
  while response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(
        expression=function_args.get("expression"),
    )

    # Step 4: send the info on the function call and function response to GPT
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    response_message = response["choices"][0]["message"]
    messages.append(response_message)  # extend conversation with assistant's reply

  for message in messages:
    print(message["role"] + ": " + message["content"])
    if message.get("function_call"):
      print(" -> calling function " + message["function_call"]["name"] + " with arguments " + json.dumps(message["function_call"]["arguments"]))
  

In [47]:
run_conversation("If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step.")

user: If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step.
assistant: To calculate your brother's age, we subtract your brother's age difference from your age. This can be represented as a mathematical expression in Latex form as "42 - 3", where 42 is your age and 3 indicates how many years your brother is younger than you.

Let's perform this calculation.
 -> calling function calc_latex_math_expression with arguments "{\n  \"expression\": \"42 - 3\"\n}"
function: 39
assistant: After performing the subtraction operation (42 - 3), the result is 39, and therefore, your brother is 39 years old.
